University of Helsinki, Master's Programme in Mathematics and Statistics  
MAST32001 Computational Statistics, Autumn 2023  
Luigi Acerbi  

# Week 2 exercises

## 1. Permutation testing (6 pts)

We will use permutation testing to study if the mother's age (`age`) affects the birth weight (`bwt`) of their babies. We will use the absolute difference in the means as the test statistic. We will focus the analysis on full term pregnancies (`gestation >= 273`).

*Note*: When reporting a $p$-value for $b$ more extreme tests out of $m$, use $p = (b+1)/(m+1)$ to avoid zero p-values. 50000 permutations will be sufficient for obtaining the required accuracy.

1. Load the data set below. Test whether the birth weights (`bwt`) of babies with young (`age < 26`) and older (`age >= 26`) mothers are statistically significantly different using the difference of the means as the test statistic. Report the $p$-value you obtain in Moodle.
2. Stratify the analysis by the variable smoking status of the mothers by splitting to separate smoker (`smoke = 0`) and non-smoker (`smoke = 1`) groups. Constrain the permutations so that only changes within each group are allowed. After the permutation, merge the two groups back together to compute the means. Report the $p$-value you obtain in Moodle.

In [1]:
import pandas as pd
import numpy as np
import numpy.random as npr
from typing import Callable
from scipy.optimize import minimize, least_squares
rng = npr.default_rng(42)
# Load the data set
babies_full = pd.read_csv("https://raw.githubusercontent.com/lacerbi/compstats-files/main/data/babies2.txt", sep='\t')

# Pick a subset
babies1 = babies_full.iloc[(babies_full['gestation']>=273).values]


In [2]:
def shuffle(x1: np.ndarray, x2: np.ndarray) -> tuple[np.ndarray, np.ndarray]:
    n = len(x1)
    joined = np.concatenate((x1, x2), axis=0).copy()
    x = rng.permutation(joined)
    return x[:n], x[n:]

In [9]:

young_mothers = babies1.query('age < 26').copy()
older_mothers = babies1.query('age >= 26').copy()

n_perm = 50000
perm_diff = np.zeros(n_perm)
true_diff = np.abs(young_mothers['bwt'].mean() - older_mothers['bwt'].mean())

# Shuffle the data
one, two = shuffle(young_mothers['bwt'].values, older_mothers['bwt'].values)
one.mean(), two.mean()
for p in range(n_perm):
    one, two = shuffle(young_mothers['bwt'].values, older_mothers['bwt'].values)
    perm_diff[p] = np.abs(one.mean() - two.mean())

print(f"Does age affect birth weights? p-value is {(np.sum(perm_diff <= true_diff)+1) / (n_perm+1)}")

Does age affect birth weights? p-value is 0.9256214875702486


In [12]:
young_smokers = babies1.query('age < 26 and smoke == 1').copy()
young_non_smokers = babies1.query('age < 26 and smoke == 0').copy()
older_smokers = babies1.query('age >= 26 and smoke == 1').copy()
older_non_smokers = babies1.query('age >= 26 and smoke == 0').copy()

n_perm = 50000
true_diff = np.abs(young_mothers['bwt'].mean() - older_mothers['bwt'].mean())
perm_diff = np.zeros(n_perm)

for p in range(n_perm):
    yns, ons = shuffle(young_non_smokers['bwt'].values, older_non_smokers['bwt'].values)
    ys, os = shuffle(young_smokers['bwt'].values, older_smokers['bwt'].values)
    smokers = np.concatenate((ys, os), axis=0)
    non_smokers = np.concatenate((yns, ons), axis=0)
    perm_diff[p] = np.abs(np.mean(smokers) - np.mean(non_smokers))

print(f"Does smoking affect birth weights?: p-value is {(np.sum(perm_diff <= true_diff)+1) / (n_perm+1)}")

Does smoking affect birth weights?: p-value is 1.999960000799984e-05


## 2. Bootstrap confidence intervals on data statistics (4 pts)

In this exercise we use bootstrap to estimate confidence intervals for various quantities. (Using 1000 bootstrap samples will give you enough accuracy assuming everything is correctly done.)

1. Use bootstrap to estimate the central 95% confidence interval for the mean of `bwt` in the *full* data set loaded in Problem 1. Report the lower and upper ends of the interval in Moodle.
2. Use bootstrap to estimate the central 95% confidence interval for the mean of `bwt` in the smaller subset (`gestation >= 273`) of the data set used in Problem 1. Report the lower and upper ends of the interval in Moodle.
3. Use bootstrap to estimate the central 95% confidence interval for the correlation coefficient of `gestation` and `age` in the full data set loaded in Problem 1. What does this tell about the relation between the duration of the pregnancy and the age of the mother? Report the bounds of the interval in Moodle.
4. Use bootstrap to estimate the central 95% confidence interval for the correlation coefficient of `gestation` and `age` in the smaller subset (`gestation >= 273`) used in Problem 1. What does this tell about the relation between the duration of the pregnancy and the age of the mother? Report the bounds of the interval in Moodle.

*Hint*: Remember that the size of the bootstrap sample is always the same as the size of the original data set.

In [35]:
n_bootstrap_samples = 1_000

#2.1
birth_weights_full = babies_full['bwt'].to_numpy()
birth_weights_full_bootstrapped_means = npr.choice(birth_weights_full, size=(n_bootstrap_samples, len(birth_weights_full)), replace=True).mean(axis=1)
birth_weights_full_mean_ci = np.quantile(birth_weights_full_bootstrapped_means, [0.025, 0.975])
print(f"95% CI for the mean birth weight of all mothers: {birth_weights_full_mean_ci}")

#2.2
birth_weights_full_term = babies1['bwt'].to_numpy()
birth_weights_full_term_bootstrapped_means = npr.choice(birth_weights_full_term, size=(n_bootstrap_samples, len(birth_weights_full_term)), replace=True).mean(axis=1)
birth_weights_full_term_mean_ci = np.quantile(birth_weights_full_term_bootstrapped_means, [0.025, 0.975])
print(f"95% CI for the mean birth weight of term mothers: {birth_weights_full_term_mean_ci}")

#2.3
corrcoef_bootstraps = np.zeros(n_bootstrap_samples)
for i in range(n_bootstrap_samples):
    age_ = npr.choice(babies_full['age'].to_numpy(), size=len(babies_full['age']), replace=True)
    gestation_ = npr.choice(babies_full['gestation'].to_numpy(), size=len(babies_full['gestation']), replace=True)
    corrcoef_bootstraps[i] = np.corrcoef(age_, gestation_)[0, 1]
print(f"95% CI for the correlation coefficient between age and gestation: {np.quantile(corrcoef_bootstraps, [0.025, 0.975])}")

#2.4
corrcoef_bootstraps = np.zeros(n_bootstrap_samples)
for i in range(n_bootstrap_samples):
    age_ = npr.choice(babies1['age'].to_numpy(), size=len(babies1['age']), replace=True)
    gestation_ = npr.choice(babies1['gestation'].to_numpy(), size=len(babies1['gestation']), replace=True)
    corrcoef_bootstraps[i] = np.corrcoef(age_, gestation_)[0, 1]
print(f"95% CI for the correlation coefficient between age and gestation: {np.quantile(corrcoef_bootstraps, [0.025, 0.975])}")


95% CI for the mean birth weight of all mothers: [118.45650043 120.55520034]
95% CI for the mean birth weight of term mothers: [122.34982759 124.67706897]
95% CI for the correlation coefficient between age and gestation: [-0.05705808  0.05747755]
95% CI for the correlation coefficient between age and gestation: [-0.06673056  0.06680831]


## 3. Bootstrap confidence intervals on parameter estimates (4 pts)

In this task, we will use bootstrap to obtain confidence intervals on maximum likelihood parameter estimates for linear regression models. We will apply simple case resampling, i.e. resampling the individuals and then fitting the model using the data $(x_i, y_i)$ from these individuals. There are alternative methods that may work better when the data are limited, but in our case there are enough observations so that this will not be a problem. 1000 bootstrap samples will again give you enough accuracy.

A linear regression fit to scalar $x_i, y_i$ involves fitting the model
$$ y_i = \alpha + \beta x_i + \epsilon_i, $$
where $\beta$ is the regression coefficient and $\alpha$ is the intercept. Assuming $\epsilon_i \sim N(0, \sigma^2)$, the log-likelihood of the model is
$$ \log p(Y | X, \alpha, \beta) = \sum_{i=1}^n \log p(y_i | x_i, \alpha, \beta)
  = \sum_{i=1}^n - \frac{1}{2 \sigma^2} (y_i - \alpha - \beta x_i)^2 + C, $$
where $C$ is independent of $\alpha, \beta$. This is maximised when
$$ \hat{\beta}= \frac{\sum_{i = 1}^n (x_i - \bar{x})(y_i - \bar{y}) }{ \sum_{i = 1}^n (x_i - \bar{x})^2} \\
   \hat{\alpha} = \bar{y} - \hat{\beta} \bar{x},$$
where $\bar{x} = \frac{1}{n} \sum_{i = 1}^n x_i$ and $\bar{y} = \frac{1}{n} \sum_{i = 1}^n y_i$.

1. Implement the above linear regression model to predict `gestation` ($y$) as a function of `age` ($x$) in the full data set. Report the estimated $\beta$ in Moodle.
2. Use bootstrap to estimate the confidence interval of the regression coefficient $\beta$ in the above model by resampling the individuals used to fit the model. Report the lower and upper bounds of the central 95% confidence interval of $\beta$ in Moodle.


In [ ]:
import pandas as pd
import numpy as np
import numpy.random as npr

# Load the data set
babies_full = pd.read_csv("https://raw.githubusercontent.com/lacerbi/compstats-files/main/data/babies2.txt", sep='\t')

babies3 = babies_full

## 4. Density estimation (6 pts)

1. Estimate the joint density of `bwt` and `age` in the full data set using kernel density estimation with a 2-dimensional Gaussian kernel
$$ K(\mathbf{x}) = \frac{1}{2\pi} \exp\left( - \frac{\|\mathbf{x}\|^2}{2} \right)
 = \frac{1}{2\pi} \exp\left( - \frac{x_1^2 + x_2^2}{2} \right) $$
using bandwidth $h=5$. Report the value of the estimated density at point `bwt=110`, `age=31` in Moodle.

*Hint*: you can verify your results by ploting a 2D histogram of the data (`matplotlib.pyplot.hist2d`) and a contour plot of the estimated density (see e.g. Sec. 5.1.1 in Course notes for a contour plot example).

2. With the Gaussian kernel above, use leave-one-out (LOO) cross validation to find the optimal $h$ in the range `np.linspace(1.0, 5.0, 50)`. The optimal $h$ maximizes the LOO log-likelihood. Report the value of $h$ and the value of the estimated density at `bwt=110`, `age=31` in Moodle.

3. Use $k$-fold cross validation with $k=17$ to find the optimal $h$ in the range `np.linspace(1.0, 5.0, 50)`. Report the value of $h$ and the value of the estimated density at `bwt=110`, `age=31` in Moodle. For this exercise, the sample point indices for the $k$ partitions of the data should consist of consecutive indices, e.g. the first partition should be the data with indices `0:69` and so on. (In practical applications it is generally good practice to randomly permute the indices as well when creating partitions, but don't do it for this exercise.)